**Goal:**
 Generate clean CSVs for Nodes (Crash, Person, LGA, SA4, State) and Relationships (WAS_INVOLVED_IN, OCCURRED_IN, PART_OF, IN_STATE) that precisely match the headers needed for LOAD CSV and reflect graph model design in arrows


## Cell 1: Setup and Configuration

In [ ]:
import pandas as pd
import os
import numpy as np
import re

# --- Configuration ---
SOURCE_FILE = os.path.join('..', 'data', 'source', 'Project2_Dataset_Corrected.csv')
OUTPUT_DIR = os.path.join('..', 'data', 'import_final') # Output directory

print(f"Source file path: {SOURCE_FILE}")
print(f"Output directory path: {OUTPUT_DIR}")
print("Setup Complete. Necessary libraries imported and paths configured.")

Source file path: ../data/source/Project2_Dataset_Corrected.csv
Output directory path: ../data/import_final
Setup Complete. Necessary libraries imported and paths configured.


## Cell 2: Load and Initial Clean Data

In [ ]:
# --- Load Raw Data ---
try:
    df_raw = pd.read_csv(SOURCE_FILE, low_memory=False)
  
    print(f"Successfully loaded {len(df_raw)} rows from {SOURCE_FILE}") 
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_FILE}. Please check the path.")
    # Stop execution if file not found
    raise

# --- Define Column Cleaning Function ---
# This function standardizes column names: lowercase, replaces non-alphanumeric with underscore
def clean_col_name(col_name):
    col_name = str(col_name)
    col_name = col_name.replace(' (', '_').replace(') ', '_').replace(' ', '_')
    clean_name = re.sub(r'[^a-zA-Z0-9_]+', '', col_name).lower()
    clean_name = clean_name.strip('_')
    return clean_name

# --- Apply Cleaning and Inspect ---
df = df_raw.copy() # Work on a copy
original_columns = df.columns.tolist()
df.columns = [clean_col_name(col) for col in df.columns]
cleaned_columns = df.columns.tolist()

print("--- Column Name Cleaning ---")
print("Original -> Cleaned:")
for orig, clean in zip(original_columns, cleaned_columns):
    if orig != clean:
        print(f"'{orig}' -> '{clean}'")
    else:
        print(f"'{orig}' -> (no change)")

print("\n--- DataFrame Info After Cleaning ---")
df.info()
print("\n--- First 5 Rows After Cleaning ---")
display(df.head())

Successfully loaded 10490 rows from ../data/source/Project2_Dataset_Corrected.csv
--- Column Name Cleaning ---
Original -> Cleaned:
'ID' -> 'id'
'Crash ID' -> 'crash_id'
'State' -> 'state'
'Month' -> 'month'
'Year' -> 'year'
'Dayweek' -> 'dayweek'
'Time' -> 'time'
'Crash Type' -> 'crash_type'
'Number Fatalities' -> 'number_fatalities'
'Bus Involvement' -> 'bus_involvement'
'Heavy Rigid Truck Involvement' -> 'heavy_rigid_truck_involvement'
'Articulated Truck Involvement' -> 'articulated_truck_involvement'
'Speed Limit' -> 'speed_limit'
'Road User' -> 'road_user'
'Gender' -> 'gender'
'Age' -> 'age'
'National Remoteness Areas' -> 'national_remoteness_areas'
'SA4 Name 2021' -> 'sa4_name_2021'
'National LGA Name 2024' -> 'national_lga_name_2024'
'National Road Type' -> 'national_road_type'
'Christmas Period' -> 'christmas_period'
'Easter Period' -> 'easter_period'
'Age Group' -> 'age_group'
'Day of week' -> 'day_of_week'
'Time of day' -> 'time_of_day'

--- DataFrame Info After Cleaning ---


,id,crash_id,state,month,year,dayweek,time,crash_type,number_fatalities,bus_involvement,...,age,national_remoteness_areas,sa4_name_2021,national_lga_name_2024,national_road_type,christmas_period,easter_period,age_group,day_of_week,time_of_day
0,1,20241115,NSW,12,2024,Friday,4:00,Single,1,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
1,2,20241125,NSW,12,2024,Friday,6:15,Single,1,No,...,19,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,17_to_25,Weekday,Day
2,3,20246013,TAS,12,2024,Friday,9:43,Single,1,No,...,33,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,26_to_39,Weekday,Day
3,4,20241002,NSW,12,2024,Friday,10:35,Single,1,No,...,32,Outer Regional Australia,New England and North West,Armidale,National or State Highway,No,No,26_to_39,Weekday,Day
4,5,20243185,QLD,12,2024,Friday,13:00,Single,1,No,...,61,Inner Regional Australia,Toowoomba,Lockyer Valley,National or State Highway,No,No,40_to_64,Weekday,Day


## Cell 3: Create Location Nodes (State, SA4, LGA)

In [19]:
print("--- Creating Location Node Files & Extracting Link Data---")

# --- States ---
states_nodes = df[['state']].drop_duplicates().dropna() # Unique states for nodes
states_nodes.rename(columns={'state': 'name:ID(State)'}, inplace=True)
states_nodes[':LABEL'] = 'State'
states_path = os.path.join(OUTPUT_DIR, 'states.csv')
states_nodes.to_csv(states_path, index=False, encoding='utf-8')
print(f"Saved {len(states_nodes)} State nodes to {states_path}")
display(states_nodes.head()) 

# --- SA4s & Link Data ---
# Extract unique SA4-State pairs first for relationship accuracy
sa4_link_data = df[['sa4_name_2021', 'state']].drop_duplicates().dropna()
# Then get unique SA4s for node file
sa4_nodes = sa4_link_data[['sa4_name_2021']].drop_duplicates().copy()
sa4_nodes.rename(columns={'sa4_name_2021': 'name:ID(SA4)'}, inplace=True)
sa4_nodes[':LABEL'] = 'SA4'
sa4s_path = os.path.join(OUTPUT_DIR, 'sa4s.csv')
sa4_nodes.to_csv(sa4s_path, index=False, encoding='utf-8')
print(f"Saved {len(sa4_nodes)} SA4 nodes to {sa4s_path}")
display(sa4_nodes.head()) 

# --- LGAs & Link Data ---

lga_link_data = df[['national_lga_name_2024', 'sa4_name_2021']].drop_duplicates().dropna()

lga_nodes = lga_link_data[['national_lga_name_2024']].drop_duplicates().copy()
lga_nodes.rename(columns={'national_lga_name_2024': 'name:ID(LGA)'}, inplace=True)
lga_nodes[':LABEL'] = 'LGA'
lgas_path = os.path.join(OUTPUT_DIR, 'lgas.csv')
lga_nodes.to_csv(lgas_path, index=False, encoding='utf-8')
print(f"Saved {len(lga_nodes)} LGA nodes to {lgas_path}")
display(lga_nodes.head()) 

print("--- Location Node Files Created & Link Data Extracted ---")

--- Creating Location Node Files & Extracting Link Data---
Saved 8 State nodes to ../data/import_final/states.csv


,name:ID(State),:LABEL
0,NSW,State
2,TAS,State
4,QLD,State
5,SA,State
73,VIC,State


Saved 88 SA4 nodes to ../data/import_final/sa4s.csv


,name:ID(SA4),:LABEL
0,Riverina,SA4
1,Sydney - Baulkham Hills and Hawkesbury,SA4
2,Launceston and North East,SA4
3,New England and North West,SA4
4,Toowoomba,SA4


Saved 509 LGA nodes to ../data/import_final/lgas.csv


,name:ID(LGA),:LABEL
0,Wagga Wagga,LGA
1,Hawkesbury,LGA
2,Northern Midlands,LGA
3,Armidale,LGA
4,Lockyer Valley,LGA


--- Location Node Files Created & Link Data Extracted ---


## Cell 4: Create Unique Crash Nodes

In [ ]:
print("--- Creating Unique Crash Node File ---") 

crash_data = df.drop_duplicates(subset=['crash_id'], keep='first').reset_index(drop=True).copy()
# --- Generate Neo4j Internal ID ---
crash_data['internalCrashID:ID(Crash)'] = crash_data.index

# --- Handle Data Types and Missing Values ---
for col in ['bus_involvement', 'heavy_rigid_truck_involvement', 'articulated_truck_involvement', 'christmas_period', 'easter_period']:
    if col in crash_data.columns:
        crash_data[col] = crash_data[col].str.lower().fillna('unknown')

crash_data['year'] = pd.to_numeric(crash_data['year'], errors='coerce').fillna(0).astype(int)
crash_data['month'] = pd.to_numeric(crash_data['month'], errors='coerce').fillna(0).astype(int)
crash_data['number_fatalities'] = pd.to_numeric(crash_data['number_fatalities'], errors='coerce').fillna(0).astype(int)
crash_data['speed_limit'] = pd.to_numeric(crash_data['speed_limit'], errors='coerce')
crash_data['speed_limit'] = crash_data['speed_limit'].astype('Int64')

# --- Select and Rename Columns for Neo4j ---
crash_node_cols = {
    'internalCrashID:ID(Crash)': 'internalCrashID:ID(Crash)',
    'crash_id': 'crashID_orig',
    'year': 'year', 'month': 'month', 'dayweek': 'dayweek', 'time': 'time',
    'crash_type': 'crashType', 'number_fatalities': 'numberFatalities', 'bus_involvement': 'busInvolvement',
    'heavy_rigid_truck_involvement': 'heavyRigidTruckInvolvement', 'articulated_truck_involvement': 'articulatedTruckInvolvement',
    'speed_limit': 'speedLimit', 'national_road_type': 'nationalRoadType', 'christmas_period': 'christmasPeriod',
    'easter_period': 'easterPeriod', 'national_remoteness_areas': 'nationalRemotenessAreas',
    'day_of_week': 'dayOfWeekType', 'time_of_day': 'timeOfDay'
}
crash_nodes = crash_data[list(crash_node_cols.keys())].copy()
crash_nodes.rename(columns=crash_node_cols, inplace=True)
crash_nodes[':LABEL'] = 'Crash'

# --- Save Crash Nodes ---
crashes_path = os.path.join(OUTPUT_DIR, 'crashes.csv')
crash_nodes.to_csv(crashes_path, index=False, encoding='utf-8')

print(f"Saved {len(crash_nodes)} unique Crash nodes to {crashes_path}") 
display(crash_nodes.head())

# --- Create Mapping for Relationships ---
# Select the original 'crash_id' (cleaned name) and the generated internal ID from crash_data
crash_id_map = crash_data[['crash_id', 'internalCrashID:ID(Crash)']].copy() # Use .copy() for safety
# Set the index using the original 'crash_id' column
crash_id_map.set_index('crash_id', inplace=True)

print("Created crash_id to internalCrashID map.") 

--- Creating Unique Crash Node File ---
Saved 9683 unique Crash nodes to ../data/import_final/crashes.csv


,internalCrashID:ID(Crash),crashID_orig,year,month,dayweek,time,crashType,numberFatalities,busInvolvement,heavyRigidTruckInvolvement,articulatedTruckInvolvement,speedLimit,nationalRoadType,christmasPeriod,easterPeriod,nationalRemotenessAreas,dayOfWeekType,timeOfDay,:LABEL
0,0,20241115,2024,12,Friday,4:00,Single,1,no,no,no,100,Arterial Road,yes,no,Inner Regional Australia,Weekday,Night,Crash
1,1,20241125,2024,12,Friday,6:15,Single,1,no,no,no,80,Local Road,no,no,Inner Regional Australia,Weekday,Day,Crash
2,2,20246013,2024,12,Friday,9:43,Single,1,no,no,no,50,Local Road,yes,no,Inner Regional Australia,Weekday,Day,Crash
3,3,20241002,2024,12,Friday,10:35,Single,1,no,no,no,100,National or State Highway,no,no,Outer Regional Australia,Weekday,Day,Crash
4,4,20243185,2024,12,Friday,13:00,Single,1,no,no,no,100,National or State Highway,no,no,Inner Regional Australia,Weekday,Day,Crash


Created crash_id to internalCrashID map.


## Cell 5: Create Person Nodes

In [17]:
print("--- Creating Person Node File ---") # Replaced logging

# --- Handle Data Types ---
df['age'] = pd.to_numeric(df['age'], errors='coerce').fillna(-1).astype(int)

# --- Select and Rename Columns for Neo4j ---
person_node_cols = {
    'id': 'personID:ID(Person)',
    'road_user': 'roadUser', 'gender': 'gender', 'age': 'age', 'age_group': 'ageGroup'
}
person_nodes = df[list(person_node_cols.keys())].copy()
person_nodes.rename(columns=person_node_cols, inplace=True)
person_nodes[':LABEL'] = 'Person'

# --- Save Person Nodes ---
persons_path = os.path.join(OUTPUT_DIR, 'persons.csv')
person_nodes.to_csv(persons_path, index=False, encoding='utf-8')

print(f"Saved {len(person_nodes)} Person nodes to {persons_path}") 
display(person_nodes.head())

--- Creating Person Node File ---
Saved 10490 Person nodes to ../data/import_final/persons.csv


,personID:ID(Person),roadUser,gender,age,ageGroup,:LABEL
0,1,Driver,Male,74,65_to_74,Person
1,2,Driver,Female,19,17_to_25,Person
2,3,Driver,Female,33,26_to_39,Person
3,4,Driver,Female,32,26_to_39,Person
4,5,Passenger,Female,61,40_to_64,Person


## Cell 6: Create Relationship Files

In [20]:
print("--- Creating Relationship Files ---")

# --- 1. SA4 -> State ---
try:
    # Use sa4_link_data created in Cell 3
    sa4_state_rels = sa4_link_data.copy()
    sa4_state_rels.rename(columns={'sa4_name_2021': ':START_ID(SA4)', 'state': ':END_ID(State)'}, inplace=True)
    sa4_state_rels[':TYPE'] = 'IN_STATE'
    rels_sa4_state_path = os.path.join(OUTPUT_DIR, 'rels_sa4_state.csv')
    sa4_state_rels.to_csv(rels_sa4_state_path, index=False, encoding='utf-8')
    print(f"Saved {len(sa4_state_rels)} SA4->State relationships to {rels_sa4_state_path}")
except Exception as e: print(f"Error in SA4->State: {e}")

# --- 2. LGA -> SA4 (Corrected Logic) ---
try:
    # Use lga_link_data created in Cell 3
    # Ensure unique mapping based on LGA name only, keeping the first SA4 found
    lga_to_sa4_unique = lga_link_data.drop_duplicates(subset=['national_lga_name_2024'], keep='first')

    # Use this unique mapping for the relationship file
    lga_sa4_rels = lga_to_sa4_unique.copy()
    lga_sa4_rels.rename(columns={'national_lga_name_2024': ':START_ID(LGA)', 'sa4_name_2021': ':END_ID(SA4)'}, inplace=True)
    lga_sa4_rels[':TYPE'] = 'PART_OF'
    rels_lga_sa4_path = os.path.join(OUTPUT_DIR, 'rels_lga_sa4.csv')
    # Drop rows if either LGA or SA4 name ended up being null after processing
    lga_sa4_rels.dropna(subset=[':START_ID(LGA)', ':END_ID(SA4)'], inplace=True)
    lga_sa4_rels.to_csv(rels_lga_sa4_path, index=False, encoding='utf-8')
    print(f"Saved {len(lga_sa4_rels)} LGA->SA4 relationships (unique per LGA) to {rels_lga_sa4_path}")
except Exception as e: print(f"Error in LGA->SA4: {e}")

# --- 3. Crash -> LGA ---
try:
    # Use crash_data (deduplicated) from Cell 4
    # Ensure source columns exist before selecting
    if 'internalCrashID:ID(Crash)' in crash_data.columns and 'national_lga_name_2024' in crash_data.columns:
        crash_lga_rels = crash_data[['internalCrashID:ID(Crash)', 'national_lga_name_2024']].copy()
        crash_lga_rels.rename(columns={'internalCrashID:ID(Crash)': ':START_ID(Crash)', 'national_lga_name_2024': ':END_ID(LGA)'}, inplace=True)
        crash_lga_rels[':TYPE'] = 'OCCURRED_IN'
        crash_lga_rels.dropna(subset=[':START_ID(Crash)', ':END_ID(LGA)'], inplace=True)
        rels_crash_lga_path = os.path.join(OUTPUT_DIR, 'rels_crash_lga.csv')
        crash_lga_rels.to_csv(rels_crash_lga_path, index=False, encoding='utf-8')
        print(f"Saved {len(crash_lga_rels)} Crash->LGA relationships to {rels_crash_lga_path}")
    else:
        print("Error in Crash->LGA: Source columns not found in crash_data.")
except Exception as e: print(f"Error in Crash->LGA: {e}")

# --- 4. Person -> Crash ---
try:
    # Use original df (with cleaned columns 'id', 'crash_id') and crash_id_map from Cell 4
    df_merged_person = df.merge(crash_id_map, left_on='crash_id', right_index=True, how='inner')
    # Ensure required columns exist after merge
    if 'id' in df_merged_person.columns and 'internalCrashID:ID(Crash)' in df_merged_person.columns:
        person_crash_rels = df_merged_person[['id', 'internalCrashID:ID(Crash)']].copy()
        person_crash_rels.rename(columns={'id': ':START_ID(Person)', 'internalCrashID:ID(Crash)': ':END_ID(Crash)'}, inplace=True)
        person_crash_rels[':TYPE'] = 'WAS_INVOLVED_IN'
        rels_person_crash_path = os.path.join(OUTPUT_DIR, 'rels_person_crash.csv')
        person_crash_rels.to_csv(rels_person_crash_path, index=False, encoding='utf-8')
        print(f"Saved {len(person_crash_rels)} Person->Crash relationships to {rels_person_crash_path}")
    else:
        print("Error in Person->Crash: Required columns ('id', 'internalCrashID:ID(Crash)') not found after merge.")
except Exception as e: print(f"Error in Person->Crash: {e}")

print("\n--- All Relationship File Generation Complete ---")

--- Creating Relationship Files ---
Saved 88 SA4->State relationships to ../data/import_final/rels_sa4_state.csv
Saved 509 LGA->SA4 relationships (unique per LGA) to ../data/import_final/rels_lga_sa4.csv
Saved 9683 Crash->LGA relationships to ../data/import_final/rels_crash_lga.csv
Saved 10490 Person->Crash relationships to ../data/import_final/rels_person_crash.csv

--- All Relationship File Generation Complete ---
